In [1]:
# ✅ Full Modified Training Script (Balanced Dataset)

import os
import json
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from PIL import Image, ImageFile

# ✅ Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ✅ Function to remove corrupted images
def remove_corrupted_images(folder_path):
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            filepath = os.path.join(subdir, file)
            try:
                img = Image.open(filepath)
                img.verify()
            except Exception:
                print(f"🗑️ Removing corrupted: {filepath}")
                os.remove(filepath)

# ✅ Function to balance dataset
def balance_class_images(folder_path, target_count):
    print(f"\n📁 Balancing images in: {folder_path}")
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_path):
            continue

        images = os.listdir(class_path)
        image_count = len(images)

        if image_count <= target_count:
            print(f"✅ '{class_name}' has {image_count} images (OK)")
        else:
            to_delete = image_count - target_count
            print(f"🗑️ '{class_name}' has {image_count} images, deleting {to_delete} extra")
            images_to_remove = random.sample(images, to_delete)
            for img in images_to_remove:
                os.remove(os.path.join(class_path, img))
            print(f"✅ '{class_name}' now has {target_count} images")

# 🛣️ Dataset paths
train_dir = r"C:\Users\Dell\Downloads\Senior-Design-VIAD-4\train"
test_dir = r"C:\Users\Dell\Downloads\Senior-Design-VIAD-4\test"

# ✅ Clean corrupted images
remove_corrupted_images(train_dir)
remove_corrupted_images(test_dir)

# ✅ Balance dataset
TARGET_COUNT_TRAIN = 70
TARGET_COUNT_TEST = 70
balance_class_images(train_dir, TARGET_COUNT_TRAIN)
balance_class_images(test_dir, TARGET_COUNT_TEST)

# 🖼️ Image settings
IMG_SIZE = (96, 96)
BATCH_SIZE = 32

# 🔄 Data generators
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)
val_data = val_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# 🏷️ Save class indices
with open("class_labels.json", "w") as f:
    json.dump(train_data.class_indices, f)

# 🧠 Build model
num_classes = train_data.num_classes
base_model = MobileNetV2(input_shape=(*IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# ⚙️ Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ⏱️ Callbacks
checkpoint = ModelCheckpoint("mobilenet_finetuned_model.keras", save_best_only=True, monitor='val_accuracy', mode='max')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 🚀 Train
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=25,
    callbacks=[checkpoint, early_stop]
)

# 💾 Save final model
model.save("mobilenet_final_model.keras")



📁 Balancing images in: C:\Users\Dell\Downloads\Senior-Design-VIAD-4\train
✅ 'Bench' has 70 images (OK)
✅ 'Bicycle' has 70 images (OK)
✅ 'Branch' has 70 images (OK)
✅ 'Bus' has 70 images (OK)
✅ 'Bushes' has 51 images (OK)
✅ 'Car' has 70 images (OK)
✅ 'Crosswalk' has 51 images (OK)
✅ 'Door' has 33 images (OK)
✅ 'Elevator' has 29 images (OK)
✅ 'Fire Hydrant' has 32 images (OK)
✅ 'Green Light' has 70 images (OK)
✅ 'Gun' has 68 images (OK)
✅ 'Motorcycle' has 40 images (OK)
✅ 'Person' has 35 images (OK)
✅ 'Pothole' has 38 images (OK)
✅ 'Rat' has 70 images (OK)
✅ 'Red Light' has 70 images (OK)
✅ 'Scooter' has 61 images (OK)
✅ 'Stairs' has 34 images (OK)
✅ 'Stop Sign' has 32 images (OK)
✅ 'Traffic Cone' has 70 images (OK)
✅ 'Train' has 49 images (OK)
✅ 'Tree' has 33 images (OK)
✅ 'Truck' has 35 images (OK)
✅ 'Umbrella' has 32 images (OK)
✅ 'Yellow Light' has 1 images (OK)

📁 Balancing images in: C:\Users\Dell\Downloads\Senior-Design-VIAD-4\test
✅ 'Bench' has 70 images (OK)
✅ 'Bicycle' has 70 

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 96, 96, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 48, 48, 32)        │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 48, 48, 32)        │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 48, 48, 32)        │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 48, 48, 32)        │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 48, 48, 32)        │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 48, 48, 32)        │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 48, 48, 16)        │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 48, 48, 16)        │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 48, 48, 96)        │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 48, 48, 96)        │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 48, 48, 96)        │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 49, 49, 96)        │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 24, 24, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,425,306 (9.25 MB)

 Trainable params: 167,322 (653.60 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

C:\Users\Dell\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
 5/41 ━━━━━━━━━━━━━━━━━━━━ 1:12 2s/step - accuracy: 0.0365 - loss: 3.8321

C:\Users\Dell\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


41/41 ━━━━━━━━━━━━━━━━━━━━ 178s 4s/step - accuracy: 0.2331 - loss: 2.9552 - val_accuracy: 0.7955 - val_loss: 0.8642
Epoch 2/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 152s 4s/step - accuracy: 0.7552 - loss: 0.8824 - val_accuracy: 0.8931 - val_loss: 0.4248
Epoch 3/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 154s 4s/step - accuracy: 0.8608 - loss: 0.4897 - val_accuracy: 0.9287 - val_loss: 0.2924
Epoch 4/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 151s 4s/step - accuracy: 0.9007 - loss: 0.3512 - val_accuracy: 0.9312 - val_loss: 0.2509
Epoch 5/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 156s 4s/step - accuracy: 0.9397 - loss: 0.2388 - val_accuracy: 0.9294 - val_loss: 0.2323
Epoch 6/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 166s 4s/step - accuracy: 0.9396 - loss: 0.1940 - val_accuracy: 0.9515 - val_loss: 0.1805
Epoch 7/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 181s 4s/step - accuracy: 0.9512 - loss: 0.1735 - val_accuracy: 0.9552 - val_loss: 0.1625
Epoch 8/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 184s 5s/step - accuracy: 0.9716 - loss: 0.1255 - val_accuracy: 0.9619 - val_loss: 0.

In [3]:
model.save("C:/Users/Dell/Downloads/webcam_model.keras")


In [7]:
import json

with open("C:/Users/Dell/Downloads/class_labels.json", "w") as f:
    json.dump(class_labels, f)


NameError: name 'class_labels' is not defined

In [ ]:
# 1. Imports
import os
import json
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

# 2. Clean corrupted images
def remove_corrupted_images(folder_path):
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            try:
                img_path = os.path.join(subdir, file)
                img = Image.open(img_path)
                img.verify()
            except Exception:
                print(f"🗑️ Removing corrupted: {img_path}")
                os.remove(img_path)

# Run for train + test
remove_corrupted_images(r"C:\Users\Dell\Downloads\Senior-Design-VIAD-4\train")
remove_corrupted_images(r"C:\Users\Dell\Downloads\Senior-Design-VIAD-4\test")

# 3. Paths
train_dir = r"C:\Users\Dell\Downloads\Senior-Design-VIAD-4\train"
val_dir = r"C:\Users\Dell\Downloads\Senior-Design-VIAD-4\test"

# 4. Config
IMG_SIZE = (96, 96)
BATCH_SIZE = 32
EPOCHS = 30

# 5. Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# 6. Save label mapping
with open("class_labels.json", "w") as f:
    json.dump(train_data.class_indices, f)

# 7. Build model
num_classes = train_data.num_classes

base_model = MobileNetV2(input_shape=(*IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# 8. Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 9. Callbacks
checkpoint_cb = ModelCheckpoint("mobilenet_best_model.keras", save_best_only=True, monitor="val_accuracy", mode="max")
early_stop_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 10. Train
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, early_stop_cb]
)

# 11. Save final model
model.save("mobilenet_final_model.keras")
